In [1]:
from langchain.pydantic_v1 import BaseModel,Field
from langchain_core.utils.function_calling import convert_to_openai_function

In [2]:
class Sentiment(BaseModel):
    """define o sentimento e o idioma da mensagem enviada"""
    sentimento:str = Field(description="Sentimento do text.deve ser 'pos','neg' ou 'nt' para nao definido")
    lingua:str = Field(description="lingua que o texto foi escrito(deve estar no charset utf-8)")


tool_sentimento = convert_to_openai_function(Sentiment)
tool_sentimento

{'name': 'Sentiment',
 'description': 'define o sentimento e o idioma da mensagem enviada',
 'parameters': {'type': 'object',
  'properties': {'sentimento': {'description': "Sentimento do text.deve ser 'pos','neg' ou 'nt' para nao definido",
    'type': 'string'},
   'lingua': {'description': 'lingua que o texto foi escrito(deve estar no charset utf-8)',
    'type': 'string'}},
  'required': ['sentimento', 'lingua']}}

In [3]:
texto = "eu gosto muito de pizza nordestina"

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

prompt = ChatPromptTemplate.from_messages([
    ("system","analise o texto e categorize-o conforme as intruções"),
    ("user","{input}")
])

chat = ChatOpenAI(api_key=os.getenv("openaiKey"))

chain = prompt | chat.bind(functions=[tool_sentimento],
                           function_call={"name":"Sentiment"})

In [6]:
chain.invoke({"input":texto})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"sentimento":"pos","lingua":"portuguese"}', 'name': 'Sentiment'}}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 131, 'total_tokens': 145, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fd12c8f-6256-496d-a3e2-b4e15ac37e0c-0')

In [ ]:
solicitacoes = [
    "Meu computador está travando toda vez que tento abrir um programa. O que devo fazer?",
    "Não consigo acessar minha conta. A senha não está funcionando!",
    "Estou tendo dificuldades para instalar o software de design gráfico. Pode me ajudar?",
    "Gostaria de saber como atualizar meu endereço de entrega na plataforma.",
    "O meu laptop não está ligando. Acho que é um problema no cabo de energia.",
    "Não consigo ver meus arquivos no aplicativo. Existe alguma solução?"
]

In [ ]:
from enum import Enum
from langchain.pydantic_v1 import BaseModel, Field

class SetorEnum(str, Enum):
    suporte_hardware = "suporte_hardware"
    suporte_software = "suporte_software"
    suporte_conta = "suporte_conta"
    outros = "outros"
    
class DirecionaSuporte(BaseModel):
    """Direciona a solicitação de suporte para o setor responsável"""
    setor: SetorEnum

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function

tool_direcionamento_suporte = convert_to_openai_function(DirecionaSuporte)
tool_direcionamento_suporte

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

# Criar o prompt com as instruções detalhadas para o modelo
system_message = '''Pense com cuidado ao categorizar o texto conforme as instruções.
Questões relacionadas a problemas de hardware, como falha no computador, laptop ou 
outros dispositivos físicos devem ser direcionadas para "suporte_hardware".
Questões relacionadas a problemas com software, como instalação, erros ao abrir programas, 
etc., devem ser direcionadas para "suporte_software".
Problemas relacionados ao acesso de conta, como recuperação de senha, problemas de login, 
devem ser direcionados para "suporte_conta".
Mensagens que não se encaixam nessas categorias devem ser direcionadas para "outros".
'''

prompt = ChatPromptTemplate.from_messages([
    ("system", system_message),
    ("user", "{input}")
])

chat = ChatOpenAI()

chain = (prompt | chat.bind(functions=[tool_direcionamento_suporte],
                            function_call={"name":"DirecionaSuporte"})
         | JsonOutputFunctionsParser())

In [ ]:
solicitacao = solicitacoes[0]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

In [ ]:
solicitacao = solicitacoes[3]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")

In [ ]:
solicitacao = solicitacoes[4]
resposta = chain.invoke({"input": solicitacao})
print(f"Solicitação: {solicitacao}")
print(f"Resposta: {resposta}")